In [24]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('customers_dataset.csv', 'customers'),
    ('orders_dataset.csv', 'orders'),
    ('order_items_dataset.csv', 'order_items'),
    ('order_reviews_dataset.csv', 'order_reviews'),
    ('order_payments_dataset.csv', 'order_payments'),
    ('products_dataset.csv', 'products'),
    ('geolocation_dataset.csv', 'geolocation'),
    ('sellers_dataset.csv', 'sellers'),
    ('product_category_name_translation.csv', 'product_category')
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Sagar19@",
    database="mercado_livre"
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = r"C:\Users\PC\Downloads\Mercado Livre - ecom\Mercado Livre - ecom"

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    print(f"\n📥 Processing file: {file_path}")
    
    try:
        # Read CSV
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"❌ Failed to read {file_path}: {e}")
        continue

    # Replace NaN with None for SQL NULL
    df = df.where(pd.notnull(df), None)

    # ✅ FIX STARTS HERE: Clean column names & drop invalids
    df.columns = [str(col).strip().replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]
    df = df.loc[:, ~df.columns.str.lower().isin(['nan', 'none', '', 'unnamed_0'])]  # drop bad headers
    df = df.loc[:, ~df.columns.duplicated()]  # drop duplicate column names
    # ✅ FIX ENDS HERE

    try:
        # Create table
        columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
        create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
        cursor.execute(create_table_query)
        print(f"✅ Table `{table_name}` created or already exists.")
    except Exception as e:
        print(f"❌ Failed to create table {table_name}: {e}")
        continue

    try:
        # Insert rows
        for _, row in df.iterrows():
            values = tuple(None if pd.isna(x) else x for x in row)
            sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
            cursor.execute(sql, values)
        conn.commit()
        print(f"✅ Inserted {len(df)} records into `{table_name}`.")
    except Exception as e:
        print(f"❌ Failed to insert data into {table_name}: {e}")

# Close connection
conn.close()
print("\n🚀 All done!")



📥 Processing file: C:\Users\PC\Downloads\Mercado Livre - ecom\Mercado Livre - ecom\customers_dataset.csv
✅ Table `customers` created or already exists.
✅ Inserted 99441 records into `customers`.

📥 Processing file: C:\Users\PC\Downloads\Mercado Livre - ecom\Mercado Livre - ecom\orders_dataset.csv
✅ Table `orders` created or already exists.
✅ Inserted 99441 records into `orders`.

📥 Processing file: C:\Users\PC\Downloads\Mercado Livre - ecom\Mercado Livre - ecom\order_items_dataset.csv
✅ Table `order_items` created or already exists.
✅ Inserted 112650 records into `order_items`.

📥 Processing file: C:\Users\PC\Downloads\Mercado Livre - ecom\Mercado Livre - ecom\order_reviews_dataset.csv
✅ Table `order_reviews` created or already exists.
✅ Inserted 99224 records into `order_reviews`.

📥 Processing file: C:\Users\PC\Downloads\Mercado Livre - ecom\Mercado Livre - ecom\order_payments_dataset.csv
✅ Table `order_payments` created or already exists.
✅ Inserted 103886 records into `order_payme